# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ribeira grande,38.5167,-28.7000,59.14,64,100,23.29,PT,1703609800
1,1,al wajh,26.2455,36.4525,78.75,50,40,9.22,SA,1703609800
2,2,khost,33.3395,69.9204,51.87,42,0,1.66,AF,1703609801
3,3,badger,64.8000,-147.5333,-5.93,80,75,0.00,US,1703609801
4,4,iqaluit,63.7506,-68.5145,0.43,92,71,4.09,CA,1703609801


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo = True,
    tiles = "OSM",
    size='Humidity',
    color='City',
    alpha=0.7,
    hover_cols=['City', 'Humidity'],
    title='City Map with Humidity'
).opts(tools=['hover'])

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] > 50) &  
    (city_data_df['Max Temp'] < 100) &
    (city_data_df['Wind Speed'] < 10) &
    (city_data_df['Cloudiness'] < 35)
]

# Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Display sample data
filtered_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,khost,33.3395,69.9204,51.87,42,0,1.66,AF,1703609801
11,11,yanam,16.7333,82.2167,71.02,85,3,4.21,IN,1703609805
18,18,ghadamis,30.1337,9.5007,61.99,24,30,8.01,LY,1703609808
22,22,ancud,-41.8697,-73.8203,65.80,62,0,7.27,CL,1703609809
27,27,montevarchi,43.5276,11.5689,54.73,94,14,2.15,IT,1703609810
...,...,...,...,...,...,...,...,...,...,...
541,541,freetown,8.4840,-13.2299,82.17,64,32,7.85,SL,1703610175
544,544,tezu,27.9167,96.1667,61.38,50,1,3.76,IN,1703610178
549,549,daxi,24.8837,121.2904,64.81,81,21,4.05,TW,1703610183
559,559,nelson bay,-32.7167,152.1500,68.05,76,2,7.11,AU,1703610193


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Display the hotel_df DataFrame
hotel_df.head()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,khost,AF,33.3395,69.9204,42,
11,yanam,IN,16.7333,82.2167,85,
18,ghadamis,LY,30.1337,9.5007,24,
22,ancud,CL,-41.8697,-73.8203,62,
27,montevarchi,IT,43.5276,11.5689,94,
...,...,...,...,...,...,...
541,freetown,SL,8.4840,-13.2299,64,
544,tezu,IN,27.9167,96.1667,50,
549,daxi,TW,24.8837,121.2904,81,
559,nelson bay,AU,-32.7167,152.1500,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radiusMeters = 10000 
params = {"categories": "accommodation.hotel", "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row["Lng"]
    lat = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{lon},{lat},{radiusMeters}"
    params["bias"] = f"proximity:{lon},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
khost - nearest hotel: No hotel found
yanam - nearest hotel: Hôtel Sarvainn
ghadamis - nearest hotel: فندق عين الفارس
ancud - nearest hotel: Cabañas
montevarchi - nearest hotel: Delta Hotel
zalingei - nearest hotel: No hotel found
arar - nearest hotel: دار أوتيل Dar Otel
isangel - nearest hotel: Tanna Lodge
raniganj - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
dikhil - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
ciudad acuna - nearest hotel: Hotel San Jorge
la rinconada - nearest hotel: Hotel Torre de los Guzmanes
georgetown - nearest hotel: Page 63 hostel
alice springs - nearest hotel: Aurora Alice Springs
khash - nearest hotel: No hotel found
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
tura - nearest hotel: No hotel found
lata - nearest hotel: Residencial Botânico
gumdag - nearest hotel: No hotel found
east london - nearest hotel: No hotel found
mukalla - nearest hotel: فند

,City,Country,Lat,Lng,Humidity,Hotel Name
2,khost,AF,33.3395,69.9204,42,No hotel found
11,yanam,IN,16.7333,82.2167,85,Hôtel Sarvainn
18,ghadamis,LY,30.1337,9.5007,24,فندق عين الفارس
22,ancud,CL,-41.8697,-73.8203,62,Cabañas
27,montevarchi,IT,43.5276,11.5689,94,Delta Hotel
...,...,...,...,...,...,...
541,freetown,SL,8.4840,-13.2299,64,Formal Leona hotel
544,tezu,IN,27.9167,96.1667,50,No hotel found
549,daxi,TW,24.8837,121.2904,81,新南12文創實驗商行
559,nelson bay,AU,-32.7167,152.1500,76,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(x= 'Lng', 
                                   y='Lat', 
                                   geo = True,
                                   tiles = "OSM",
                                   size='Humidity',
                                   color='City',
                                   alpha=0.7,
                                   hover_cols = ['City', 'Humidity', 'Hotel Name', 'Country'],
                                   title='Hotel Map with Humidity'
                                  ).opts(tools=['hover'])

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)